In [17]:
import numpy as np
import pandas as pd

In [18]:
comp = pd.read_csv('data/composite-database.csv')

In [19]:
comp.sort_values('elo', ascending = False).head(10)
comp = comp.drop(columns = ['Unnamed: 0'])
comp.head(5)

,id,name,id_opp,name_opp,event_id,event_name,elo,elo_opp,height_diff,reach_diff,...,str_def_diff,td_avg_diff,td_acc_diff,td_def_diff,sub_avg_diff,wins_diff,losses_diff,momentum_diff,wl_diff_diff,outcome
0,429e7d3725852ce9,Royce Gracie,63b65af1c5cb02cb,Ken Shamrock,6420efac0578988b,UFC 1: The Beginning,750.0,750.000000,0.0,3.0,...,-2.0,-0.94,-60.0,22.0,-1.7,5.0,-7.0,5.0,12.0,W
1,63b65af1c5cb02cb,Ken Shamrock,429e7d3725852ce9,Royce Gracie,6420efac0578988b,UFC 1: The Beginning,750.0,750.000000,0.0,-3.0,...,2.0,0.94,60.0,-22.0,1.7,-5.0,7.0,-5.0,-12.0,L
2,429e7d3725852ce9,Royce Gracie,08ae5cd9aef7ddd3,Kimo Leopoldo,1a49e0670dfaca31,UFC 3: The American Dream,766.0,750.000000,-2.0,0.0,...,7.0,-4.55,-100.0,66.0,-1.5,10.0,-3.0,3.0,13.0,W
3,08ae5cd9aef7ddd3,Kimo Leopoldo,429e7d3725852ce9,Royce Gracie,1a49e0670dfaca31,UFC 3: The American Dream,750.0,766.000000,2.0,-0.0,...,-7.0,4.55,100.0,-66.0,1.5,-10.0,3.0,-3.0,-13.0,L
4,63b65af1c5cb02cb,Ken Shamrock,429e7d3725852ce9,Royce Gracie,dedc3bb440d09554,UFC 5: The Return of the Beast,734.0,781.263693,0.0,-3.0,...,2.0,0.94,60.0,-22.0,1.7,-5.0,7.0,-5.0,-12.0,W


In [89]:
comp = pd.read_csv('data/composite-database.csv')
train = pd.read_csv('data/composite-train.csv')

In [90]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
    #Fit the model:
    model.fit(data[predictors],data[outcome])
  
    #Make predictions on training set:
    predictions = model.predict(data[predictors])
  
    #Print accuracy
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print("Accuracy : %s" % "{0:.3%}".format(accuracy))

    #Perform k-fold cross-validation with 10 folds
    kf = KFold(n_splits=10, shuffle=False)
    error = []
    for train, test in kf.split(data):
        # Filter training data
        train_predictors = (data[predictors].iloc[train,:])
    
        # The target we're using to train the algorithm.
        train_target = data[outcome].iloc[train]
    
        # Training the algorithm using the predictors and target.
        model.fit(train_predictors, train_target)
    
        #Record error from each cross-validation run
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

    #Fit the model again so that it can be refered outside the function:
    model.fit(data[predictors],data[outcome]) 

In [91]:
predictors = ['elo','elo_opp','height_diff','reach_diff','ss_min_diff','str_acc_diff', 
            'str_a_min_diff','str_def_diff','td_avg_diff','td_acc_diff','td_def_diff','sub_avg_diff', 
            'wins_diff','losses_diff','momentum_diff','wl_diff_diff']
outcome = 'outcome'
model = LogisticRegression()
classification_model(model, train, predictors, outcome)

Accuracy : 75.728%
Cross-Validation Score : 75.195%


In [92]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classification_model(model, train, predictors, outcome)

Accuracy : 100.000%
Cross-Validation Score : 67.881%


In [93]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=30)
classification_model(model, train, predictors, outcome)

Accuracy : 99.811%
Cross-Validation Score : 76.639%


In [112]:
test = pd.read_csv('data/composite-test.csv')
results = pd.read_csv('data/test-results.csv')
test = test.iloc[::2]
test = test.reset_index()
results = results.iloc[::2]
results = results.reset_index()

In [117]:
# Logistic Regression Testing
model = LogisticRegression()
model.fit(train[predictors],train[outcome])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 89.93%


In [114]:
# Decision Tree Testing
model = DecisionTreeClassifier()
model.fit(train[predictors],train[outcome])
predictions = model.predict(train[predictors])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 72.48%


In [115]:
# Random Forest Testing
model = RandomForestClassifier()
model.fit(train[predictors],train[outcome])
predictions = model.predict(train[predictors])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 79.87%
